# URL 피싱 탐지 모델 학습 + Hugging Face
## Google Colab + Hugging Face Hub 통합

**기능:**
- Random Forest 모델 학습
- Hugging Face Hub에 모델 업로드
- Hugging Face Datasets에서 피싱 URL 로드 (선택)

**작성자:** 이경준

## 1. 환경 설정 + Hugging Face

In [ ]:
# 필요한 라이브러리 설치
!pip install tldextract python-whois scikit-learn pandas numpy joblib
!pip install huggingface_hub datasets  # Hugging Face 추가

In [ ]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse
import tldextract
from collections import Counter
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
import joblib

# Hugging Face
from huggingface_hub import HfApi, create_repo, upload_file, login
from datasets import load_dataset, Dataset

print("✅ 라이브러리 로드 완료")

## 2. Hugging Face 로그인 (선택)

모델을 Hugging Face Hub에 업로드하려면 로그인이 필요합니다.

**토큰 발급:**
1. https://huggingface.co/settings/tokens
2. "New token" > Write 권한 선택
3. 토큰 복사

In [ ]:
# Hugging Face 로그인 (모델 업로드 시 필요)
from google.colab import userdata
import os

# 방법 1: Colab Secrets 사용 (추천)
# Colab > 설정 (왼쪽 열쇠 아이콘) > Secrets > HF_TOKEN 추가
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("✅ Hugging Face 로그인 완료")
except:
    # 방법 2: 직접 입력
    print("⚠️ Colab Secrets에 HF_TOKEN이 없습니다.")
    print("직접 로그인하려면 아래 주석을 해제하세요:")
    # login()  # 대화형 로그인

## 3. 데이터셋 로드

### 옵션 A: 샘플 데이터 (빠른 테스트)

In [ ]:
# 샘플 데이터 생성 (기존 코드 동일)
malicious_urls = [
    "http://paypal-secure-login.com/verify",
    "http://amazon-account-verify.com/update",
    "http://apple-id-locked.com/unlock",
    "http://netflix-payment-failed.com/update",
    "http://google-security-alert.com/verify",
    "http://microsoft-account-suspended.com/restore",
    "http://facebook-unusual-activity.com/verify",
    "http://instagram-verify-account.com/confirm",
    "http://cj-delivery-notice.com/parcel",
    "http://hanjin-package-arrived.com/check",
    "http://192.168.1.100/admin/login",
    "http://bit.ly/free-iphone-giveaway",
    "http://secure-banking-update-urgent.com",
    "http://verify-your-account-now-2024.com",
    "http://claim-your-reward-immediately.com",
    "http://paypa1-secure123.com",
]

benign_urls = [
    "https://www.google.com",
    "https://www.youtube.com",
    "https://www.facebook.com",
    "https://www.amazon.com",
    "https://www.wikipedia.org",
    "https://www.naver.com",
    "https://www.github.com",
    "https://www.stackoverflow.com",
]

df = pd.DataFrame({
    'url': malicious_urls + benign_urls,
    'label': [1] * len(malicious_urls) + [0] * len(benign_urls)
})

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ 샘플 데이터셋: {len(df)}개")
print(f"   - 악성: {(df['label']==1).sum()}개")
print(f"   - 정상: {(df['label']==0).sum()}개")

### 옵션 B: Hugging Face Datasets에서 로드 (실제 데이터)

In [ ]:
# Hugging Face에서 피싱 URL 데이터셋 로드 (예시)
# 주의: 실제로 사용 가능한 데이터셋을 검색해서 사용하세요

try:
    # 예시: phishing-url-detection 데이터셋
    # dataset = load_dataset("username/phishing-urls")
    # df = dataset['train'].to_pandas()
    
    print("ℹ️ Hugging Face Datasets 사용 시 이 셀을 수정하세요")
    print("   검색: https://huggingface.co/datasets?search=phishing")
except:
    print("⚠️ 데이터셋 로드 실패 - 샘플 데이터를 사용합니다")

## 4-8. 특징 추출 및 모델 학습

(이전 노트북과 동일 - 특징 추출, 데이터 분할, 학습, 평가)

In [ ]:
# [이전 노트북의 특징 추출 함수 코드 동일]
# ... (생략)

## 9. 모델을 Hugging Face Hub에 업로드 🤗

In [ ]:
# 1. 로컬에 모델 저장 (기존)
joblib.dump(model, 'url_classifier.pkl')
joblib.dump(list(X.columns), 'feature_names.pkl')

metadata = {
    'model_type': 'RandomForest',
    'n_features': len(X.columns),
    'accuracy': acc,
    'precision': prec,
    'recall': rec,
    'f1_score': f1,
    'trained_on': 'Google Colab',
    'framework': 'scikit-learn',
    'task': 'binary-classification',
    'labels': ['benign', 'malicious']
}
joblib.dump(metadata, 'metadata.pkl')

print("✅ 로컬 저장 완료")

# 2. Hugging Face Hub에 업로드
HF_USERNAME = "your-username"  # 본인의 Hugging Face username
REPO_NAME = "url-phishing-detector"
REPO_ID = f"{HF_USERNAME}/{REPO_NAME}"

try:
    # Repository 생성 (이미 있으면 스킵)
    api = HfApi()
    
    try:
        create_repo(REPO_ID, repo_type="model", exist_ok=True)
        print(f"✅ Repository 생성: {REPO_ID}")
    except:
        print(f"ℹ️ Repository 이미 존재: {REPO_ID}")
    
    # 파일 업로드
    files_to_upload = [
        'url_classifier.pkl',
        'feature_names.pkl',
        'metadata.pkl'
    ]
    
    for file in files_to_upload:
        upload_file(
            path_or_fileobj=file,
            path_in_repo=file,
            repo_id=REPO_ID,
            repo_type="model"
        )
        print(f"   ✅ {file} 업로드 완료")
    
    print(f"\n🎉 모델 업로드 완료!")
    print(f"📍 모델 URL: https://huggingface.co/{REPO_ID}")
    
except Exception as e:
    print(f"⚠️ 업로드 실패: {e}")
    print("   로그인 확인 또는 username 수정이 필요할 수 있습니다.")

## 10. README.md 생성 (Model Card)

In [ ]:
# Model Card 생성
readme_content = f"""
---
language: en
license: mit
tags:
- phishing-detection
- url-classification
- security
- sklearn
---

# URL Phishing Detector

## Model Description

Random Forest 모델로 URL의 악성 여부를 판단하는 이진 분류기입니다.

**프로젝트:** 스미싱 탐지 시스템 (OCR/QR + URL 위험 판단)  
**작성자:** 이경준

## Performance

- **Accuracy:** {acc:.2%}
- **Precision:** {prec:.2%}
- **Recall:** {rec:.2%}
- **F1-Score:** {f1:.2%}

## Features

30개의 URL 특징을 사용합니다:
- URL 구조 (길이, 특수문자, 하이픈 등)
- 도메인 특성 (IP 주소, 엔트로피, TLD 등)
- 콘텐츠 특징 (HTTPS, 의심 키워드, 브랜드 불일치 등)

## Usage

```python
import joblib
from huggingface_hub import hf_hub_download

# 모델 다운로드
model_path = hf_hub_download(repo_id="{REPO_ID}", filename="url_classifier.pkl")
model = joblib.load(model_path)

# 예측
# features = extract_features(url)  # 특징 추출 필요
# prediction = model.predict([features])
```

## Training Data

- Phishing URLs: PhishTank, URLhaus
- Benign URLs: Tranco Top Sites

## Limitations

- 새로운 피싱 패턴에 대한 지속적인 재학습 필요
- 단축 URL은 확장 후 분석 권장

## Citation

```
@misc{{url-phishing-detector,
  author = {{이경준}},
  title = {{URL Phishing Detector}},
  year = {{2025}},
  publisher = {{Hugging Face}},
  howpublished = {{\\url{{https://huggingface.co/{REPO_ID}}}}}
}}
```
"""

# README 저장
with open('README.md', 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("✅ README.md 생성 완료")

# README 업로드
try:
    upload_file(
        path_or_fileobj='README.md',
        path_in_repo='README.md',
        repo_id=REPO_ID,
        repo_type="model"
    )
    print("✅ README.md 업로드 완료")
except:
    print("⚠️ README 업로드 실패 (선택사항)")

## 11. 다운로드 (선택)

Hugging Face에 업로드했으므로 직접 다운로드는 선택사항입니다.

In [ ]:
# 로컬로 다운로드 (선택)
from google.colab import files

print("💾 파일 다운로드 (선택):")
# files.download('url_classifier.pkl')
# files.download('feature_names.pkl')
# files.download('metadata.pkl')

print("\nℹ️ Hugging Face Hub에서 직접 다운로드 가능:")
print(f"   https://huggingface.co/{REPO_ID}/tree/main")